<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a768e7237525cb942a49c444b4ca59cbea81717e8c596ece5b95109d29d56965
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-29 11:19:36
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.46 C
-------------------
Today PnL: 65.56 K (0.45%)
Current PnL: -18.70 L (-12.26%)
CY Booked + Current PnL: -6.57 L (-4.31%)
-------------------
Total profit:  2.67 L
Total loss:  -21.37 L
-------------------
Total Booked + Current PnL: 20.41 L (16.25%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.34%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 86.40 L (59.38%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.00,-7.28,73.0,X-LC,3.84,231956.0,21590.0,7191.0,0.19,10.26,3.10,13.68,37.0,3.00,1.61,28.48,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.00,111.74,66.0,M-SC,1.42,91268.0,-9509.0,9601.0,-2.72,-9.44,10.52,0.09,245.0,-0.99,0.63,19.91,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-9.64,40.0,X-MC,2.75,89175.0,-1503.0,16408.0,-0.15,-1.66,18.40,16.44,101.0,-0.09,0.62,18.18,X40,ATH,PHARMA
42,ITC,452.00,-36.66,63.0,X-LC,1.42,204402.0,4264.0,16618.0,0.02,2.13,8.13,10.44,4.0,0.26,1.42,7.81,X40,NTT,FMCG
34,ICICIGI,2252.93,-12.91,65.0,X-MC,4.26,149643.0,13510.0,17074.0,0.16,9.92,11.41,22.47,91.0,0.79,1.04,24.19,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,53.97,68.0,M-MC,10.15,242854.0,17892.0,149768.0,6.71,7.95,61.67,74.53,192.0,0.12,1.69,42.77,XY24,BTT,STEEL
82,VAIBHAVGBL,521.00,65.84,69.0,H-SC,3.82,143566.0,-39209.0,150271.0,6.10,-21.45,104.67,60.77,125.0,-0.26,1.00,28.59,XR,NTT,JEWELLERY
57,RECLTD,446.00,47.87,60.0,M-LC,5.47,209468.0,6298.0,35840.0,3.23,3.10,17.11,20.74,55.0,0.18,1.45,10.17,XY25,NTT,FINANCE
84,VBL,671.64,-17.33,56.0,X-LC,9.19,297786.0,-18056.0,130728.0,2.77,-5.72,43.90,35.67,5.0,-0.14,2.07,7.35,X40N,ATH,FMCG
81,UNITDSPR,1644.00,-8.95,63.0,X-LC,4.92,238512.0,4368.0,44244.0,2.58,1.87,18.55,20.77,86.0,0.10,1.66,8.91,X40N,NTT,BREWERIES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,STARHEALTH,761.00,14.16,46.0,H-SC,12.49,254483.0,-26640.0,159510.0,-5.43,-9.48,62.68,47.26,144.0,-0.17,1.77,36.35,XY24,NTT,INSURANCE
17,CAMS,5211.76,-5.83,47.0,X-SC,0.40,107117.0,5113.0,38808.0,-4.00,5.01,36.23,43.06,122.0,0.13,0.74,22.99,X40N,ATH,MISC
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88818.0,-39852.0,85150.0,-2.81,-30.97,95.87,35.20,219.0,-0.47,0.62,1.94,X40,NTT,FOOTWEAR
79,TTKPRESTIG,770.00,111.74,66.0,M-SC,1.42,91268.0,-9509.0,9601.0,-2.72,-9.44,10.52,0.09,245.0,-0.99,0.63,19.91,OX40N,NTT,DURABLES
63,SAMMAANCAP,326.00,-195.45,70.0,M-SC,29.56,164034.0,14814.0,129374.0,-2.67,9.93,78.87,96.62,208.0,0.11,1.14,77.52,XY25,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,WIPRO,420.0,-14.7,45.0,M-LC,5.73,150059.0,-886.0,110338.0,-0.14,-0.59,73.53,72.51,53.0,-0.01,1.04,5.4,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,111.74,66.0,M-SC,1.42,91268.0,-9509.0,9601.0,-2.72,-9.44,10.52,0.09,245.0,-0.99,0.63,19.91,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.24,52.0,H-SC,2.72,225477.0,-44190.0,80518.0,0.41,-16.39,35.71,13.47,138.0,-0.55,1.57,14.65,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-22.53,53.0,H-MC,6.77,104852.0,-26983.0,69213.0,-0.48,-20.47,66.01,32.03,98.0,-0.39,0.73,17.44,OX40N,NTT,IT
68,SIS,528.0,2044.56,47.0,H-SC,3.14,86547.0,-24485.0,48094.0,-0.75,-22.05,55.57,21.26,156.0,-0.51,0.60,16.93,OX40N,NTT,MISC
18,CERA,9475.0,-24.82,38.0,H-SC,2.66,136988.0,-38915.0,80933.0,-0.18,-22.12,59.08,23.89,149.0,-0.48,0.95,18.31,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,53.97,68.0,M-MC,10.15,242854.0,17892.0,149768.0,6.71,7.95,61.67,74.53,192.0,0.12,1.69,42.77,XY24,BTT,STEEL
57,RECLTD,446.00,47.87,60.0,M-LC,5.47,209468.0,6298.0,35840.0,3.23,3.10,17.11,20.74,55.0,0.18,1.45,10.17,XY25,NTT,FINANCE
32,HINDZINC,730.22,28.52,51.0,M-LC,11.35,209692.0,4616.0,107949.0,2.30,2.25,51.48,54.89,52.0,0.04,1.46,26.89,X5K,ATH,METALS
47,KPIGREEN,731.05,11.32,73.0,H-SC,5.38,132703.0,7406.0,51515.0,0.76,5.91,38.82,47.03,141.0,0.14,0.92,66.16,MH,ATH,POWER
80,UNIONBANK,163.00,-8.53,62.0,M-LC,9.39,165468.0,24628.0,19856.0,-1.05,17.49,12.00,31.59,66.0,1.24,1.15,48.61,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-16.77,73.0,H-MC,16.20,215581.0,35773.0,76639.0,-0.65,19.90,35.55,62.52,88.0,0.47,1.50,60.76,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-37.89,65.0,M-SC,3.37,128726.0,13928.0,80119.0,1.30,12.13,62.24,81.92,223.0,0.17,0.89,52.39,XR,NTT,DURABLES
47,KPIGREEN,731.05,11.32,73.0,H-SC,5.38,132703.0,7406.0,51515.0,0.76,5.91,38.82,47.03,141.0,0.14,0.92,66.16,MH,ATH,POWER
37,INDIAMART,4810.62,-53.15,56.0,H-SC,6.79,130672.0,7336.0,124295.0,-0.60,5.95,95.12,106.72,119.0,0.06,0.91,30.72,AR,ATH,MISC
32,HINDZINC,730.22,28.52,51.0,M-LC,11.35,209692.0,4616.0,107949.0,2.30,2.25,51.48,54.89,52.0,0.04,1.46,26.89,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,18.69,149374.0,-127149.0,237221.0,0.35,-45.98,158.81,39.81,54.0,-0.54,1.04,5.28,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88818.0,-39852.0,85150.0,-2.81,-30.97,95.87,35.20,219.0,-0.47,0.62,1.94,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-30.55,38.0,X-MC,2.57,201150.0,330.0,67466.0,0.09,0.16,33.54,33.76,80.0,0.00,1.40,5.37,X40,ATH,FMCG
24,DMART,5391.45,-16.74,38.0,X-LC,9.89,101808.0,-778.0,27590.0,0.50,-0.76,27.10,26.13,38.0,-0.03,0.71,24.62,X40N,ATH,FMCG
31,HINDUNILVR,2922.00,-11.32,40.0,X-LC,7.61,248920.0,-8143.0,43287.0,-0.32,-3.17,17.39,13.67,24.0,-0.19,1.73,16.60,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-2.18,49.0,H-LC,1.06,157285.0,-28810.0,76283.0,1.16,-15.48,48.50,25.51,15.0,-0.38,1.09,15.91,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,63.0,X-LC,1.42,204402.0,4264.0,16618.0,0.02,2.13,8.13,10.44,4.0,0.26,1.42,7.81,X40,NTT,FMCG
53,PGHH,17907.65,-30.55,38.0,X-MC,2.57,201150.0,330.0,67466.0,0.09,0.16,33.54,33.76,80.0,0.00,1.40,5.37,X40,ATH,FMCG
29,HAVELLS,2069.16,-5.47,50.0,X-MC,3.03,315525.0,-12466.0,118984.0,1.48,-3.80,37.71,32.48,92.0,-0.10,2.19,7.19,X40,ATH,ELECTRICAL
59,RELIANCE,1533.00,-7.28,73.0,X-LC,3.84,231956.0,21590.0,7191.0,0.19,10.26,3.10,13.68,37.0,3.00,1.61,28.48,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88818.0,-39852.0,85150.0,-2.81,-30.97,95.87,35.20,219.0,-0.47,0.62,1.94,X40,NTT,FOOTWEAR
10,BAJAJHFL,181.50,-14.48,47.0,X-MC,5.94,180384.0,-20122.0,117286.0,0.90,-10.04,65.02,48.45,90.0,-0.17,1.25,4.02,X40N,ATH,FINANCE
52,PAGEIND,51769.93,-27.30,41.0,X-MC,6.80,82090.0,-1420.0,21450.0,0.32,-1.70,26.13,23.98,82.0,-0.07,0.57,4.02,X40,ATH,APPARELS
21,COLPAL,3767.14,-3.66,49.0,X-MC,7.87,225140.0,-38225.0,151564.0,0.84,-14.51,67.32,43.04,84.0,-0.25,1.56,4.52,XY25,ATH,FMCG
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,18.69,149374.0,-127149.0,237221.0,0.35,-45.98,158.81,39.81,54.0,-0.54,1.04,5.28,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-45.45,50.0,X-SC,38.22,54115.0,-10891.0,161820.0,0.35,-16.75,299.03,232.18,198.0,-0.07,0.38,5.55,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.30,41.0,X-MC,6.80,82090.0,-1420.0,21450.0,0.32,-1.70,26.13,23.98,82.0,-0.07,0.57,4.02,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.20,47.0,X-SC,4.80,82927.0,-62233.0,138165.0,1.53,-42.87,166.61,52.31,136.0,-0.45,0.58,11.00,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.64,40.0,X-MC,2.75,89175.0,-1503.0,16408.0,-0.15,-1.66,18.40,16.44,101.0,-0.09,0.62,18.18,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,4.75,88818.0,-39852.0,85150.0,-2.81,-30.97,95.87,35.20,219.0,-0.47,0.62,1.94,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.96,-28.04,55.0,X-LC,11.31,287264.0,-58692.0,125391.0,-0.06,-16.97,43.65,19.28,1.0,-0.47,1.99,6.20,X40,ATH,IT
40,INFY,2275.00,-17.44,60.0,X-LC,6.14,322120.0,9232.0,162445.0,0.81,2.95,50.43,54.87,3.0,0.06,2.24,11.48,X40,BTT,IT
42,ITC,452.00,-36.66,63.0,X-LC,1.42,204402.0,4264.0,16618.0,0.02,2.13,8.13,10.44,4.0,0.26,1.42,7.81,X40,NTT,FMCG
84,VBL,671.64,-17.33,56.0,X-LC,9.19,297786.0,-18056.0,130728.0,2.77,-5.72,43.90,35.67,5.0,-0.14,2.07,7.35,X40N,ATH,FMCG
1,ABB,7934.00,-39.48,55.0,H-LC,5.80,252816.0,-8803.0,128026.0,1.77,-3.36,50.64,45.57,7.0,-0.07,1.76,7.58,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7155.56,65.0,L-SC,7.63,79628.0,-14182.0,90800.0,0.39,-15.12,114.03,81.67,269.0,-0.16,0.55,54.20,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,50.0,L-SC,27.94,81600.0,-31949.0,72004.0,-0.21,-28.14,88.24,35.27,268.0,-0.44,0.57,99.72,XR,NTT,HOTELS
50,MASFIN,398.61,-20.51,45.0,H-SC,9.14,90465.0,-7515.0,29121.0,-0.05,-7.67,32.19,22.05,152.0,-0.26,0.63,31.50,XR,ATH,FINANCE
79,TTKPRESTIG,770.00,111.74,66.0,M-SC,1.42,91268.0,-9509.0,9601.0,-2.72,-9.44,10.52,0.09,245.0,-0.99,0.63,19.91,OX40N,NTT,DURABLES
86,WHIRLPOOL,2270.00,-37.89,65.0,M-SC,3.37,128726.0,13928.0,80119.0,1.30,12.13,62.24,81.92,223.0,0.17,0.89,52.39,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SONACOMS,806.63,-31.93,71.0,M-SC,7.97,84656.0,-16503.0,56508.0,-0.12,-16.31,66.75,39.54,202.0,-0.29,0.59,19.37,AR,ATH,AUTO
47,KPIGREEN,731.05,11.32,73.0,H-SC,5.38,132703.0,7406.0,51515.0,0.76,5.91,38.82,47.03,141.0,0.14,0.92,66.16,MH,ATH,POWER
63,SAMMAANCAP,326.00,-195.45,70.0,M-SC,29.56,164034.0,14814.0,129374.0,-2.67,9.93,78.87,96.62,208.0,0.11,1.14,77.52,XY25,NTT,FINANCE
12,BANKINDIA,190.00,-16.77,73.0,H-MC,16.20,215581.0,35773.0,76639.0,-0.65,19.90,35.55,62.52,88.0,0.47,1.50,60.76,XR,NTT,BANKS
5,ANGELONE,3033.00,14.80,60.0,X-SC,5.45,203565.0,12559.0,45151.0,-2.18,6.58,22.18,30.21,157.0,0.28,1.41,28.83,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.51
1,25,44.00
2,50,74.94


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.02
LC    32.34
MC    24.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.11
X40      16.26
XY25     12.05
XR       11.46
X40N     10.59
AR        7.84
OX40N     6.51
X200      1.76
X5K       1.46
SR        1.08
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.42
X-LC    21.61
X-MC    18.78
M-SC    12.65
M-LC     5.10
H-LC     4.61
X-SC     4.50
H-MC     3.89
M-MC     1.69
L-SC     1.45
L-LC     1.02
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.48,-3.40,37.14
IT,12.50,-16.80,79.45
FINANCE,12.09,-8.56,57.77
MISC,6.86,-16.44,80.96
ELECTRICAL,5.81,-7.97,46.24
BANKS,5.67,-4.90,66.87
PAINTS,5.49,-16.68,34.08
INSURANCE,3.75,-2.39,40.57
AUTO,2.77,-44.31,104.54


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3240000.0,22
XR,1298532.0,14
AR,1161228.0,9
X40,815554.0,12
X40N,635418.0,9
OX40N,546329.0,9
XY25,446227.0,8
SR,266264.0,2
X5K,107949.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3318481.0,24
M-SC,1223581.0,15
X-MC,1171621.0,14
X-LC,1028086.0,13
X-SC,532830.0,6
H-MC,320110.0,3
H-LC,275576.0,3
M-LC,273983.0,4
L-SC,252106.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1159088.0      6
           AR         833179.0      5
           XR         764673.0      7
M-SC       XY24       699218.0      6
X-MC       XY24       571445.0      4
X-LC       X40        446493.0      5
           XY24       304547.0      2
X-SC       X40N       285860.0      4
X-MC       X40        283911.0      6
H-SC       SR         266264.0      2
           OX40N      243762.0      3
H-LC       AR         204309.0      2
X-LC       X40N       202562.0      3
H-MC       XY24       174258.0      1
X-MC       XY25       169269.0      2
L-SC       XR         162804.0      2
X-SC       XY24       161820.0      1
M-MC       XY24       149768.0      1
X-MC       X40N       146996.0      2
M-SC       OX40N      144052.0      4
           XY25       129374.0      1
           XR         127197.0      2
           AR         123740.0      2
M-LC       XR         110338.0      1
           X5K        107949.0      1
L-SC       OX40N       89302.0      1
X-SC       X40         85150.0      1
H-MC       XR          76639.0      1
X-LC       XY25        74484.0      3
H-LC       X200        71267.0      1
H-MC       OX40N       69213.0      1
L-MC       XR          56881.0      1
H-SC       MH          51515.0      1
L-LC       XY25        37260.0      1
M-LC       XY25        35840.0      1
           XY24        19856.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
